# LangChain API Tutorial

This tutorial demonstrates how to use LangChain's core functionalities.
LangChain is a powerful framework designed to facilitate building language model-powered applications.
We'll explore its components, including prompt creation, chains, retrieval, and agents.

## Setting Up

We'll start by setting up the environment and initializing our language model (`ChatOpenAI`).
Here, we're using OpenAI's `gpt-4o-mini` model with a temperature of 0 for deterministic outputs.

In [1]:
#!sudo /venv/bin/pip install langchain --quiet
#!sudo /venv/bin/pip install -U langchain-community --quiet
#!sudo /venv/bin/pip install -U langchain-openai --quiet
#!sudo /venv/bin/pip install --quiet chromadb

In [2]:
import logging
import os
import random
import time

import helpers.hdbg as hdbg
import langchain
import langchain.agents as lngchagents
import langchain.document_loaders.csv_loader as csvloader
import langchain.prompts as lngchprmt
import langchain.schema.messages as lnchscme
import langchain.schema.runnable as lngchschrun
import langchain_community.vectorstores as vectorstores
import langchain_core.output_parsers as lngchoutpar
import langchain_openai as lngchopai

In [3]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

INFO  > cmd='/venv/lib/python3.12/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-8c2761ac-1946-448c-8889-66f927fb8c54.json'


## Add your OpenAPI key and initiate GPT model

In [4]:
# Add OpenAPI to environment variable.
os.environ["OPENAI_API_KEY"] = "your-openapi-key"
# Initiate OpenAI model.
chat_model = lngchopai.ChatOpenAI(model="gpt-4o-mini", temperature=0)

## Message Handling with `HumanMessage` and `SystemMessage`

LangChain uses `HumanMessage` and `SystemMessage` objects to structure conversations.
- **`SystemMessage`**: Defines the behavior of the assistant.
- **`HumanMessage`**: Represents user input.

Let's see how this works in practice with some example messages.

In [5]:
# Define system behavior and user input.
messages = [
    lnchscme.SystemMessage(
        content="You're an assistant knowledgeable about healthcare."
    ),
    lnchscme.HumanMessage(content="What is Medicaid managed care?"),
]
# Generate a response.
chat_model.invoke(messages)

INFO  HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='Medicaid managed care is a system of delivering Medicaid services through private health insurance plans. In this model, state Medicaid programs contract with private managed care organizations (MCOs) to provide a range of healthcare services to Medicaid beneficiaries. Here are some key features of Medicaid managed care:\n\n1. **Care Coordination**: MCOs are responsible for coordinating care for their members, which can help improve health outcomes and reduce unnecessary hospitalizations.\n\n2. **Cost Control**: Managed care aims to control costs by negotiating rates with providers and managing the utilization of services. This can help states manage their Medicaid budgets more effectively.\n\n3. **Access to Services**: Beneficiaries typically have access to a network of providers, which may include primary care physicians, specialists, hospitals, and other healthcare services. Members may need to choose a primary care provider (PCP) and get referrals for specialist 

## Restricting Assistant's Scope

You can further control the assistant's responses by tailoring the `SystemMessage`.
For instance, we'll restrict it to only answer healthcare-related questions.

In [6]:
messages = [
    SystemMessage(
        content="You're an assistant knowledgeable about healthcare. Only answer healthcare-related questions."
    ),
    HumanMessage(content="How do I change a tire?"),
]
chat_model.invoke(messages)

INFO  HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content="I'm here to assist with healthcare-related questions. If you have any health concerns or questions about medical topics, feel free to ask!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 31, 'total_tokens': 58, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_27322b4e16', 'id': 'chatcmpl-BFBXd4JH6r1a3ypqnuZNj4nL4Iari', 'finish_reason': 'stop', 'logprobs': None}, id='run-129daa36-0ed7-4448-affc-87502c23c89d-0', usage_metadata={'input_tokens': 31, 'output_tokens': 27, 'total_tokens': 58, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Creating Custom Prompts with `ChatPromptTemplate`

Custom prompts are essential for tasks like summarization, question answering, or content generation.
We'll use `ChatPromptTemplate` to define structured prompts and format them dynamically.

In [9]:
# Define a custom template for hospital reviews.
review_template_str = """
Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed as possible, but don't make up
any information that's not from the context. If you don't know an answer, say you don't know.

{context}

{question}
"""

review_template = lngchprmt.ChatPromptTemplate.from_template(review_template_str)

# Provide context and a question.
context = "I had a great stay!"
question = "Did anyone have a positive experience?"

# Format the template.
print(review_template.format(context=context, question=question))

Human: 
Your job is to use patient reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed as possible, but don't make up
any information that's not from the context. If you don't know an answer, say you don't know.

I had a great stay!

Did anyone have a positive experience?



## Advanced Prompt Structuring

Sometimes, you may need to structure prompts with multiple components, like system instructions and user input.
`SystemMessagePromptTemplate` and `HumanMessagePromptTemplate` allow fine-grained control.

In [11]:
# Define system and human message templates.
review_system_prompt = lngchprmt.SystemMessagePromptTemplate(
    prompt=lngchprmt.PromptTemplate(
        input_variables=["context"], template=review_template_str
    )
)

review_human_prompt = lngchprmt.HumanMessagePromptTemplate(
    prompt=lngchprmt.PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

# Combine into a chat prompt template.
review_prompt_template = lngchprmt.ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=[review_system_prompt, review_human_prompt],
)

# Format messages.
formatted_messages = review_prompt_template.format_messages(
    context=context, question=question
)
print(formatted_messages)

[SystemMessage(content="\nYour job is to use patient reviews to answer questions about their experience at a hospital.\nUse the following context to answer questions. Be as detailed as possible, but don't make up\nany information that's not from the context. If you don't know an answer, say you don't know.\n\nI had a great stay!\n\nDid anyone have a positive experience?\n", additional_kwargs={}, response_metadata={}), HumanMessage(content='Did anyone have a positive experience?', additional_kwargs={}, response_metadata={})]


## Chains

Chains are a fundamental abstraction in LangChain, allowing you to combine prompts and models into workflows.
We'll create a simple chain to answer questions based on a given context.

In [12]:
# Create a chain using the template and chat model.
output_parser = lngchoutpar.StrOutputParser()
review_chain = review_prompt_template | chat_model | output_parser

# Test the chain.
review_chain.invoke({"context": context, "question": question})

INFO  HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'Yes, one patient mentioned that they had a great stay at the hospital, indicating a positive experience.'

In [30]:
!ls /shared_data/dev/

anidhi	  ck_marketing	danya	jsmeriga  pavolr    sameepp  tomaz
chutianm  dan		grisha	ninat	  samarthk  sonaalk  vedanshuj


## Retrieval with FAISS

FAISS (Facebook AI Similarity Search) is used to efficiently retrieve relevant documents based on similarity scores.
We'll demonstrate how to load a dataset, create embeddings, and retrieve documents.

In [16]:
#!cp /Users/saggese/src/github/langchain_neo4j_rag_app/data/reviews.csv build_LLM_RAG_chatbot_with_langchain/.
REVIEWS_CSV_PATH = "build_LLM_RAG_chatbot_with_langchain/reviews.csv"
REVIEWS_CHROMA_PATH = "chroma_data"

# Load reviews dataset.
loader = csvloader.CSVLoader(file_path=REVIEWS_CSV_PATH, source_column="review")
reviews = loader.load()

# Create a vector store.
reviews_vector_db = vectorstores.Chroma.from_documents(
    reviews, lngchopai.OpenAIEmbeddings(), persist_directory=REVIEWS_CHROMA_PATH
)

# Retrieve relevant documents.
question = "Has anyone complained about communication with the hospital staff?"
relevant_docs = reviews_vector_db.similarity_search(question, k=3)

print(relevant_docs[0].page_content)
print(relevant_docs[1].page_content)

cp: cannot stat '/Users/saggese/src/github/langchain_neo4j_rag_app/data/reviews.csv': No such file or directory


RuntimeError: Error loading build_LLM_RAG_chatbot_with_langchain/reviews.csv

## Building a Retrieval-Enhanced QA Chain

We'll combine retrieval and prompts to build a more dynamic question-answering system.
This chain retrieves relevant documents and uses them as context for the assistant.

In [ ]:
# Create a retriever.
reviews_retriever = reviews_vector_db.as_retriever(k=10)

# Build the QA chain.
review_chain = (
    {"context": reviews_retriever, "question": lngchschrun.RunnablePassthrough()}
    | review_prompt_template
    | chat_model
    | lngchoutpar.StrOutputParser()
)

# Test the QA chain.
question = "Has anyone complained about communication with the hospital staff?"
review_chain.invoke(question)

## Agents

Agents are more flexible than chains, as they can decide the sequence of actions to take.
We'll demonstrate an agent that can answer questions using tools for reviews and wait times.

- The chain is hardwired
- An agent is an LLM that decides the sequence of actions to execute.

In [ ]:
def get_current_wait_time(hospital: str) -> int | str:
    """
    Dummy function to generate fake wait times.

    :param
    """
    if hospital not in ["A", "B", "C", "D"]:
        return f"Hospital {hospital} does not exist"
    # Simulate API call delay.
    time.sleep(1)
    return random.randint(0, 10000)

In [ ]:
# Tool is an interface that an agent uses to interact with a function.
# Each description explains the Agent when to call each tool.
tools = [
    lngchagents.Tool(
        name="Reviews",
        func=review_chain.invoke,
        description="""Useful when you need to answer questions
        about patient reviews or experiences at the hospital.
        Not useful for answering questions about specific visit
        details such as payer, billing, treatment, diagnosis,
        chief complaint, hospital, or physician information.
        Pass the entire question as input to the tool. For instance,
        if the question is "What do patients think about the triage system?",
        the input should be "What do patients think about the triage system?"
        """,
    ),
    lngchagents.Tool(
        name="Waits",
        func=get_current_wait_time,
        description="""Use when asked about current wait times
        at a specific hospital. This tool can only get the current
        wait time at a hospital and does not have any information about
        aggregate or historical wait times. This tool returns wait times in
        minutes. Do not pass the word "hospital" as input,
        only the hospital name itself. For instance, if the question is
        "What is the wait time at hospital A?", the input should be "A".
        """,
    ),
]

hospital_agent_prompt = langchain.hub.pull("hwchase17/openai-functions-agent")

agent_chat_model = ChatOpenAI(
    model="gpt-3.5-turbo-1106",
    temperature=0,
)

hospital_agent = lngchagents.create_openai_functions_agent(
    llm=agent_chat_model,
    prompt=hospital_agent_prompt,
    tools=tools,
)

# Agent run-time.
hospital_agent_executor = lngchagents.AgentExecutor(
    agent=hospital_agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True,
)

In [ ]:
hospital_agent_executor.invoke(
    {"input": "What is the current wait time at hospital C?"}
)

In [ ]:
hospital_agent_executor.invoke(
    {"input": "What have patients said about their comfort at the hospital?"}
)